# determine genre of the movies
## natural process langueges on imdb movies

### Sara Zahedi

In [1]:
import random 
import tqdm
import nltk, re
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
import nltk

In [3]:
import json
f = open('dataset.json')
data = json.load(f)
# for i in data[0:5]:
#     print(i)
f.close()

### get number of data

In [4]:
print("number of movies data:",len(data))

number of movies data: 1878


### count all storyline words and senteces

In [5]:
words_counter = 0
sentence_counter = 0

for d in data:
    for sl in d["storyline"]:
        num_words = sl.split()
        words_counter += len(num_words)
        num_sentence = sl.split(". ")
        sentence_counter += len(num_sentence)
print("total storyline words:", words_counter)
print("total storyline tences:", sentence_counter)

total storyline words: 177601
total storyline tences: 8962


In [11]:
import numpy as np
import string
import nltk

from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

### normalize data

get and remove stopwords

In [12]:
stopwords = set(stopwords.words('english'))
# stopwords

In [13]:
def remove_stopwords(text):
    text = text.lower()
    return [x for x in nltk.word_tokenize(text) if x not in stopwords and x not in string.punctuation]

cleaned_tokens = [remove_stopwords(' '.join([str(elem) for elem in x['storyline']])) for x in data]


In [14]:
tokens_len = len(cleaned_tokens)
print("number of tokens:",tokens_len)

number of tokens: 1878


Stemm tokens

In [15]:
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

lemmatizer = WordNetLemmatizer()
stemmer  = PorterStemmer()

l = len(cleaned_tokens)
for i in range(l):
    elment_len = len(cleaned_tokens[i])
    for w in range(elment_len):
        cleaned_tokens[i][w] = (stemmer.stem(cleaned_tokens[i][w]))
# cleaned_tokens

lemmatizing tokens

In [16]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

for i in range(l):
    elment_len = len(cleaned_tokens[i])
    for w in range(elment_len):
        cleaned_tokens[i][w] = (lemmatizer.lemmatize(cleaned_tokens[i][w]))

# cleaned_tokens

In [17]:
tokens_len = len(cleaned_tokens)
for i in range(tokens_len):
    cleaned_tokens[i] = [item for item in cleaned_tokens[i] if len(item)>3]

In [24]:
print("print some tokens:\n" , cleaned_tokens[0:3][0:3])

print some tokens:
 [['riddler', 'sadist', 'serial', 'killer', 'begin', 'murder', 'polit', 'figur', 'gotham', 'batman', 'forc', 'investig', 'citi', 'hidden', 'corrupt', 'question', 'famili', 'involv', 'riddler', 'sadist', 'serial', 'killer', 'begin', 'murder', 'polit', 'figur', 'gotham', 'batman', 'forc', 'investig', 'citi', 'hidden', 'corrupt', 'question', 'famili', 'involv', 'riddler', 'sadist', 'serial', 'killer', 'begin', 'murder', 'polit', 'figur', 'gotham', 'batman', 'forc', 'investig', 'citi', 'hidden', 'corrupt', 'question', 'famili', 'involv'], ['visionari', 'director', 'robert', 'egger', 'come', 'northman', 'action-fil', 'epic', 'follow', 'young', 'vike', 'princ', 'quest', 'aveng', 'father', 'murder', 'visionari', 'director', 'robert', 'egger', 'come', 'northman', 'action-fil', 'epic', 'follow', 'young', 'vike', 'princ', 'quest', 'aveng', 'father', 'murder', 'visionari', 'director', 'robert', 'egger', 'come', 'northman', 'action-fil', 'epic', 'follow', 'young', 'vike', 'princ

In [25]:
def extend_story(tokens):
    return tokens + [lemmatizer.lemmatize(x) for x in tokens] + [stemmer.stem(x) for x in tokens]

cleaned_tokens = [extend_story(x) for x in cleaned_tokens]
len(cleaned_tokens)

1878

#### labelling words

In [26]:
inv_words = dict(enumerate(set(sum(cleaned_tokens, []))))
dict_words = dict((y, x) for x, y in inv_words.items())

In [43]:
# print("print some dict words:", dict_words)

#### labelling genres

In [34]:
inv_label = dict(enumerate(set(sum([x['genre'] for x in data], []))))
dict_label = dict((y, x) for x, y in inv_label.items())

In [ ]:
# print("print some dict label:", dict_label)

In [46]:
dim_words = len(dict_words)
dim_level = len(dict_label)
print("dim words", dim_words)
print("dim level", dim_level)

dim words 12717
dim level 27


create vectors

In [40]:
vectors = []
for row in data:
    story, label = row['storyline'], row['genre']
    vector = [0] * dim_words
    story = ' '.join(map(str, story)).lower()

    tokens = [dict_words.get(x) for x in nltk.word_tokenize(story)]
    tokens = [x for x in tokens if x]
    labels = [dict_label.get(x) for x in label]
    labels = [x for x in labels if x]
    
    for ind in tokens:
        vector[ind] += 1
    
    vector = np.array(vector)
    vector = vector / np.linalg.norm(vector)
    labels = np.array(labels)
    
    vector = [vector, labels]
    vectors.append(vector)

In [39]:
print(vectors[0:1])
len(vectors)

[[array([0., 0., 0., ..., 0., 0., 0.]), array([10, 14])]]


1878

##

### Give a storyline and get the genre

In [47]:
text = "John Rambo is offered the chance to cut short his prison tenure by going on a secret mission for the US government deep into the jungles of Vietnam and locating American POWs."
text = "Set in Albuquerque, New Mexico, between 2008 and 2010, Breaking Bad follows Walter White, a meek high school chemistry teacher who transforms into a ruthless player in the local methamphetamine drug trade, driven by a desire to financially provide for his family after being diagnosed with terminal lung cancer"
threshold = 0.15

def perdict(_text, _vectors):
    
    tokens = remove_stopwords(_text.lower())

    tokens = [dict_words.get(x) for x in extend_story(tokens)]
    tokens = [x for x in tokens if x]
#     print("tokens" + tokens[0])
    
    inp_vector = [0] * dim_words
    
    for ind in tokens:
        inp_vector[ind] += 1
    
    inp_vector = np.array(inp_vector)
    inp_vector = inp_vector / np.linalg.norm(inp_vector)
    
    labels = []
    for vec, lab in _vectors:
        res = np.dot(vec, inp_vector)
        if res >= 1 - threshold:
            labels.extend(lab)
    return list(dict.fromkeys([inv_label[x] for x in labels]))

genre_len = len(perdict(text, vectors))

while(genre_len == 0):
    if (genre_len > 0.95):
        print("not found!")
        break
    threshold += 0.05
    genre_len = len(perdict(text, vectors))
    
print(perdict(text, vectors))

['Drama', 'Thriller']
